# Installation and setup

1. The first step is to connect to the SQLite database via Python. 
2. This would allow me to use this Jupyter notebook + so that I can see the results of my queries in a neatly formatted table.
3. To do this, I will
    i) define a function that takes the query (stored as a string) as an input
    ii)And shows the result as a formatted dataframe:

In [83]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
db = sqlite3.connect('hubway.db')
def run_query(query):
    return pd.read_sql_query(query,db)

In [20]:
query = 'SELECT * FROM trips LIMIT 5;'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


# Q1: What was the duration of the longest and shortest trip?

1.1 Longest 5 trips: Interesting that they are all from 'Casual'/non-registered customers and taken between 2011-2012. 

Hypothesis: If you are aware of the pricing of a product you frequently use, you will do your best to ensure that you are cost conscious.

Further investigation: 
i) Are "Registered" users more aware of the extra fees for long trips causing them NOT to take them?
ii) Can Hubway do a better job of conveying their pricing structure to Casual users to help them avoid overage charges? 

In [22]:
query = 'SELECT * FROM trips Order by duration DESC LIMIT 5'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,541247,9999,2012-09-26 18:34:00,54,2012-09-26 21:21:00,54,T01078,Casual,,,
1,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
2,416523,9998,2012-08-15 12:11:00,54,2012-08-15 14:58:00,80,B00188,Casual,,,
3,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
4,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,


1.2: Shortest 5 trips: Interesting that they are all MALE and REGISTERED and that they took their trips in August,2011. 
Further investigation: 
i) What exactly was happening in 25th August 2011 that made MALES in zipcode '02125' in between stations '38' and '40 ' take short rides on Bluebikes? 

In [24]:
query = 'SELECT * FROM trips Order by duration ASC LIMIT 5'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,11586,0,2011-08-09 14:31:00,44,2011-08-09 14:31:00,44,B00430,Registered,'02446,1956.0,Male
1,19554,0,2011-08-16 17:26:00,38,2011-08-16 17:26:00,38,B00572,Registered,'02114,1969.0,Male
2,31868,0,2011-08-25 08:55:00,40,2011-08-25 08:55:00,40,B00248,Registered,'02125,1983.0,Male
3,32195,0,2011-08-25 13:38:00,38,2011-08-25 13:38:00,38,B00366,Registered,'02125,1983.0,Male
4,32196,0,2011-08-25 13:39:00,38,2011-08-25 13:39:00,38,B00328,Registered,'02125,1983.0,Male


# Question 2 & 3: What was the ratio between registered and "casual"/unregistered users?

In [29]:
query= 'SELECT COUNT(*) FROM trips WHERE sub_type="Registered"'
run_query(query)

,COUNT(*)
0,1105192


In [30]:
query= 'SELECT COUNT(*) FROM trips WHERE sub_type="Casual"'
run_query(query)

,COUNT(*)
0,464809


In [44]:
query = '''
SELECT COUNT (*)
FROM trips
GROUP BY sub_type;
'''
run_query(query)

,COUNT (*)
0,464809
1,1105192


This also answers the trips that were taken by 'registered' users vs. 'casual' users. 


# Question 4: What was the average trip duration taken ?

Time to explore with some aggregate functions: SUM, MAX, COUNT etc. 

In [43]:
query= 'SELECT AVG(duration)/60 AS "Average duration in minutes" FROM trips'
run_query(query)

,Average duration in minutes
0,15.206828


Some context to back this figure: Hubway charges extra fees for trips over 30 minutes as the service is designed for riders to take short, one-way trips.How about we explore more and do this study on the different user groups?

# Question 5: Do registered or casual users take longer trips?

In [47]:
query = '''
SELECT sub_type, AVG(duration)/60 AS "Average Duration in minutes"
FROM trips
GROUP BY sub_type;
'''
run_query(query)

,sub_type,Average Duration in minutes
0,Casual,25.327398
1,Registered,10.950434


Note the difference babes!!
On average, registered users take trips that last around 11 minutes whereas casual users are spending almost 25 minutes per ride. 

Hypothesis: Registered users are likely taking shorter, more frequent trips, possibly as part of their commute to work and casual users are spending around twice as long per trip (probably tourists).

# Question 6: Which bike was used for the most trips? (and why)

In [52]:
query = '''
SELECT bike_number, count(*) as "Number of trips" 
FROM trips GROUP BY bike_number 
ORDER BY count(*) 
DESC LIMIT 1;
''' 
run_query(query)

,bike_number,Number of trips
0,B00490,2120


# Question 7: What is the average duration of trips by registered members over the age of 30? 

In [72]:
query = '''
SELECT AVG(duration)/60 AS 'Average duration of trips in minutes'
FROM trips
WHERE (2020 - birth_date) > 30;
''' 
run_query(query)

,Average duration of trips in minutes
0,15.275237


# ??! I tried to start the data analysis with Python because I wanted to share some visuals but I hit a roadblock. Below is an attempt of me trying to FORGE through. I have since gotten a little impatient because it's 2 am on Monday morning and I need to go to sleep. Let's try this another day :(

In [93]:
# Create a connection instance
myConnection = sqlite3.connect('hubway.db')

# create a Cursor object
myCursor = myConnection.cursor()

# Store the query as string
myQuery = 'SELECT * FROM trips LIMIT 5'

# Query the dataset
myCursor.execute(myQuery)

# Fetch the results 
Bike_sharing = myCursor.execute(myQuery).fetchall()
print(Bike_sharing)

# Close the connection
myConnection.close()

[(1, 9, '2011-07-28 10:12:00', 23, '2011-07-28 10:12:00', 23, 'B00468', 'Registered', "'97217", 1976.0, 'Male'), (2, 220, '2011-07-28 10:21:00', 23, '2011-07-28 10:25:00', 23, 'B00554', 'Registered', "'02215", 1966.0, 'Male'), (3, 56, '2011-07-28 10:33:00', 23, '2011-07-28 10:34:00', 23, 'B00456', 'Registered', "'02108", 1943.0, 'Male'), (4, 64, '2011-07-28 10:35:00', 23, '2011-07-28 10:36:00', 23, 'B00554', 'Registered', "'02116", 1981.0, 'Female'), (5, 12, '2011-07-28 10:37:00', 23, '2011-07-28 10:37:00', 23, 'B00554', 'Registered', "'97214", 1983.0, 'Female')]
